In [31]:
!nvidia-smi

Fri Nov 26 18:22:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 455.38       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3080    Off  | 00000000:01:00.0  On |                  N/A |
| 53%   31C    P8    31W / 340W |   3049MiB / 10018MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Work in Progress

This is an implementation of RGCNN from the paper https://github.com/tegusi/RGCNN

Initial testing will be done on the GeometricShapes dataset from PyG

In [1]:
import torch
import torch_geometric
'''
from torch_geometric.datasets import GeometricShapes
dataset = GeometricShapes(rood='data/GeometricShapes')
'''
from torch_geometric.datasets import ModelNet

dataset = ModelNet(root='data/ModelNet', train=True)
dataset_test = ModelNet(root='data/ModelNet', train=False)
print("The whole dataset:       \t", dataset)
print("Example from the dataset:\t", dataset[0])

The whole dataset:       	 ModelNet10(3991)
Example from the dataset:	 Data(pos=[1777, 3], face=[3, 2509], y=[1])


Converting the Meshes from the dataset to a PointCloud:

In [2]:
from torch_geometric.transforms import SamplePoints
from torch_geometric.transforms import NormalizeScale
from torch_geometric.transforms import Compose

# For reproductibility
torch.manual_seed(42)
points_number = 300  #For quicker test. Default: 1024
dataset.transform = Compose([SamplePoints(num=points_number), NormalizeScale()])
dataset_test.transform = Compose([SamplePoints(num=points_number), NormalizeScale()])
#dataset.transform = NormalizeScale()
#dataset_test.transform = NormalizeScale()
print("Example of Train Point Cloud: \t", dataset[0])
print("Example of Test Point Cloud: \t", dataset_test[0])
# print("Example of Test Point CloudValues : \t", dataset_test[0].pos)


Example of Train Point Cloud: 	 Data(pos=[300, 3], y=[1])
Example of Test Point Cloud: 	 Data(pos=[300, 3], y=[1])


The following classes are defined as in the RGCNN github:

In [3]:
import torch.nn as nn

class GetGraph(nn.Module):
    def __init__(self):
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

    def diag(self, mat):
        # input is batch x vertices
        d = []
        for vec in mat:
            d.append(torch.diag(vec))
        return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=2)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.matrix_diag(D)
            L = D - adj_matrix
        return L

Testing the functions:

In [ ]:
get_graph = GetGraph()
print("Shape of the dataset points: \t", dataset[0].pos.shape)
data=dataset[0].pos
data=data.reshape([1,data.shape[0], data.shape[1]])
print("New shape:                   \t", data.shape)
x=data
W = get_graph(data)

W_torch = W
print("Graph shape:                 \t", W_torch.shape)
W=torch.squeeze(W)
print("Reduction of the data dimension:\t", W.shape)
print("Graph:                       \t", W)


getter_laplacian = GetLaplacian()
L_torch = getter_laplacian(W_torch)
L = torch.squeeze(L_torch)
print("Laplacian shape:             \t", L_torch.shape)
print("Squeezed Laplacian shape     \t", L.shape)
print("Laplacian                    \t", L)


Defining the Chebyshev approximation:

In [40]:
import torch.nn as nn
def find_eigmax(L):
    with torch.no_grad():
        e1, _ = torch.eig(L, eigenvectors=False)
        return torch.max(e1[:, 0]).item()

def chebyshev_Lapl(X, Lapl, thetas, order):
    list_powers = []
    nodes = Lapl.shape[0]

    T0 = X.float()

    eigmax = find_eigmax(Lapl)
    L_rescaled = (2 * Lapl / eigmax) - torch.eye(1024)

    y = T0 * thetas[0]
    list_powers.append(y)
    T1 = torch.matmul(L_rescaled, T0)
    list_powers.append(T1 * thetas[1])

    # Computation of: T_k = 2*L_rescaled*T_k-1  -  T_k-2
    for k in range(2, order):
        T2 = 2 * torch.matmul(L_rescaled, T1) - T0
        list_powers.append((T2 * thetas[k]))
        T0, T1 = T1, T2
    y_out = torch.stack(list_powers, dim=-1)
    # the powers may be summed or concatenated. i use concatenation here
    y_out = y_out.view(nodes, -1) # -1 = order* features_of_signal
    return y_out

Testing the Chebyshev approx:

In [ ]:
features_1 = 3          # input features number for the 'model'
out_features_1 = 128    # output features number from the first convolutional layer
K1 = 6                  # order of the Chebyshev Polynomial
thetas_1 = nn.Parameter(torch.rand(points_number))  # initialization of the parameters of the model for the first conv layer
out = chebyshev_Lapl(x,L,thetas_1,K1)           # computing the approximate polynomial
print('cheb approx out powers concatenated:', out.shape)    
linear_1 = nn.Linear(K1*features_1, out_features_1)     # MLP == Linear
layer_out_1 = linear_1(out)
print('Layers output:', layer_out_1.shape)

Here I will try to define a convolutional layer:

In [4]:
import torch_geometric.nn.conv as cnv
import torch_geometric.utils as tgu

class RGCNN_Conv(nn.Module):
    def __init__(self, F_in, K, F_out):
        super(RGCNN_Conv, self).__init__()
        self.F_in = F_in
        self.K = K
        self.F_out = F_out
        '''
        self.W = nn.Parameter(torch.Tensor(self.K * self.F_in, self.F_out))
        nn.init.normal_(self.W, mean=0, std=0.2)

        self.B = nn.Parameter(torch.Tensor(self.F_out))
        nn.init.normal_(self.B, mean=0, std=0.2)
        '''
        self.cheb_conv = cnv.ChebConv(in_channels=F_in, out_channels=F_out, K=K, normalization="sym", bias=True)

        self.relu = nn.ReLU()

    def forward(self, x, W):
        N, M, F_in = list(x.size())
        edge_index, edge_weight = tgu.dense_to_sparse(W)
        x = self.cheb_conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
        x = self.relu(x)
        return x.reshape(N, M, self.F_out)


TESTS:

In [ ]:
x = dataset[0].pos
x = x.reshape(1, x.shape[0], x.shape[1])
get_graph = GetGraph()
W = get_graph(x)
edge_index, edge_weight = tgu.dense_to_sparse(W)
conv1 = RGCNN_Conv(F_in=3, K=6, F_out=128)
aux = conv1(x, W)
#print(aux.shape)

print(conv1.state_dict())   # This shows the Parameters of the Module

With the defined Convolutional Layer we can create our model:

In [25]:
class RGCNN_Model(nn.Module):
    def __init__(self, points_number, F, K, M, in_features):

        # Verify the consistency w.r.t. the number of layers.
        assert len(F) == len(K)
        
        super(RGCNN_Model, self).__init__()
        self.points_number = points_number
        self.F = F
        self.K = K
        self.M = M
        self.layers_number = len(F)
        self.get_graph = GetGraph()
        self.convolution_list = []
        self.mlp_list = []
        self.rloss = nn.MSELoss()
        self.pool = nn.MaxPool1d(self.points_number)
        self.relu = nn.ReLU()

        for i in range(self.layers_number):
            if i == 0:
                layer = RGCNN_Conv(in_features, K[i], F[i])
            else:
                layer = RGCNN_Conv(F[i-1], K[i], F[i])
            setattr(self, 'gcn%d' % i, layer)

        for i in range(len(M)):
            if i == 0:
                mlp = nn.Linear(F[-1], M[i])
            else:
                mlp = nn.Linear(M[i-1], M[i])
            setattr(self, 'mlp%d' % i, mlp )

    def forward(self, x, label):
        # W = self.get_graph(x)
        losses = []
        #labels = torch.zeros(40)
        #labels[label] = 1
        #labels = labels.to(device)
        for i in range(self.layers_number):
            W = self.get_graph(x)
            x = getattr(self, 'gcn%d' % i)(x, W)
        # loss = self.rloss(x, labels)
        x = x.permute(0, 2, 1)
        x = self.pool(x)
        x.squeeze_(2)
        for i in range(len(self.M)):
            x = getattr(self, 'mlp%d' % i)(x)
            x = self.relu(x)
        labels = torch.zeros(x.shape[-1])
        labels[label] = 1
        labels = labels.reshape(shape=list(x.shape))
        labels = labels.to(device)
        loss = self.rloss(x, labels)
        return x, loss

In [26]:
F = [128, 512, 1024, 512, 128, 40]  # Number of graph convolutional filters.
K = [6, 5, 3, 1, 1, 1]  # Polynomial orders.
M = [512, 128, 10]  # Output dimensionality of fully connected layers.
criterion = torch.nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print("Training on ", device)
model = RGCNN_Model(points_number=points_number, F=F, K=K, M=M, in_features=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Training on  cuda


In [27]:
rloss = nn.MSELoss()
def train(train_loader):
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        X = data.pos
        label = data.y
        X = X.reshape([1, X.shape[0], X.shape[1]])
        X = X.to(device)
        label = label.to(torch.float)
        label = label.to(device)
        # Forward pass.
        # print(label.to(torch.int).item())
        out, _ = model(X, label.to(torch.int).item())
        pred = out.argmax(dim=-1)
        loss = rloss(pred, label)
        # Calculate gradients.
        # loss.backward()
        # Updates the models parameters
        optimizer.step()

print(list(dataset[0].pos.size()))

#train(dataset)

[300, 3]


In [28]:
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        X = data.pos
        labels = data.y
        X, labels = X.to(device), labels.to(device)  
        X = X.reshape(1, X.shape[0], X.shape[1])
        # Forward pass.
        out, loss = model(X, labels)
        # Take the index of the class with the highest probability.
        pred = out.argmax(dim=1)
        # Compare with ground-truth labels.
        correct += int((pred == labels).sum()) 
    return correct / len(loader)

# test(dataset_test)

In [29]:
for epoch in range(1, 101):
    train(dataset)
    train_acc = test(dataset)
    print(f'Epoch: {epoch}, Train Acc: {train_acc:.4f}')

Epoch: 1, Train Acc: 0.0501
Epoch: 2, Train Acc: 0.0501
Epoch: 3, Train Acc: 0.0501
Epoch: 4, Train Acc: 0.0501
Epoch: 5, Train Acc: 0.0501
Epoch: 6, Train Acc: 0.0501
Epoch: 7, Train Acc: 0.0501
Epoch: 8, Train Acc: 0.0501
Epoch: 9, Train Acc: 0.0501


KeyboardInterrupt: 

In [ ]:
print(model.state_dict())

In [23]:
X = dataset[0].pos
labels = dataset[0].y
X, labels = X.to(device), labels.to(device)  
X = X.reshape(1, X.shape[0], X.shape[1])
out, loss = model(X, labels)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<ReluBackward0>)
